In [ ]:
from legion.metrics import Metric
import legion.model
import legion.pymodel.store

STORAGE = legion.pymodel.store.SharedStore('summation-store')

In [ ]:
legion.model.init('test summation', '1.0')
legion.model.define_property('number.pow', 1)
legion.model.define_property('number.pow_of_ten', 1)

STORAGE.state_pow = legion.model.properties.get('number.pow_of_ten', cast=legion.model.int32)


def on_new_properties():    
    new_value = legion.model.properties.get('number.pow_of_ten', cast=legion.model.int32)
    print('Updating global state variable to {}'.format(new_value))
    STORAGE.STATE_POW = new_value
        
                
legion.model.on_property_update(on_new_properties)

In [ ]:
def calculate_sum(x):
    return int(x['a']) + int(x['b'])


def calculate_sum_and_pow(x):
    return (10 ** STORAGE.state_pow) * ((x['a'] + x['b']) ** legion.model.properties.get('number.pow', cast=legion.model.int32))

In [ ]:
legion.model.send_metric(Metric.TEST_ACCURACY, 100.0)

In [ ]:
legion.model.export_untyped(lambda x: {'result': int(calculate_sum(x))})

legion.model.export(lambda x: {'result': int(calculate_sum_and_pow(x))}, {
    'a': legion.model.int32,
    'b': legion.model.int32
}, endpoint='sum_and_pow')

legion.model.save()